## Choose our model

In [1]:
import pickle
model = pickle.load(open('clf_LGBM.pickle', 'rb'))

## Localised Test

In [9]:
import pandas as pd
PCD = pd.read_csv("pcd_FR_mean_ethnic.csv")

Data_Set = 1

# we need to load the entire postcode file as we will use it as a lookup on the PostCode to the attributes to feed into our model..

postcodes = []
#postcodes.append('SS141JB')
postcodes.append('EN87BW')
postcodes.append('CB11PA')
postcodes.append('CM11PH')
postcodes.append('CM20BX')
postcodes.append('LU14LH')
postcodes.append("BB97AG")
postcodes.append("B81HN")
postcodes.append("BB95HY")
postcodes.append("L339TW")
postcodes.append("BB16EA")
postcodes.append("BS494HS")

test_BTI = []
test_RVM_sect = []
test_RVM_area = []
test_RVM_dist = []
test_White = []
test_Mixed = []
test_Asian = []
test_BAC = []
test_Other = []
test_Deprivation = []
test_GeoCode_Lat = []
test_GeoCode_Long = []

for postcode in postcodes:
    row = PCD.loc[PCD['PostCode_S'] == postcode]
   # print (row)
    row2 = row.to_numpy()
    test_BTI.append ("1")
    test_RVM_sect.append(row2[:,3])
    test_RVM_area.append(row2[:,4])
    test_RVM_dist.append(row2[:,5])
    test_White.append(row2[:,6])
    test_Mixed.append(row2[:,7])
    test_Asian.append(row2[:,8])
    test_BAC.append(row2[:,9])
    test_Other.append(row2[:, 10])
    test_Deprivation.append(row2[:,12])

if Data_Set ==1:
    test_data = pd.DataFrame(list(zip(test_BTI, test_RVM_sect, test_RVM_area, test_RVM_dist, test_White, test_Mixed, test_Asian, test_BAC, test_Other, test_Deprivation)), columns = ['Business Type Index', 'RVM_Sector', 'RVM_Area', 'RVM_Dist', 'White', 'Mixed', 'Asian', 'BAC', 'Other', 'Deprivation'])
elif Data_Set == 2:
    test_data = pd.DataFrame(list(zip(test_White, test_Mixed, test_Asian, test_BAC, test_Other, test_Deprivation)), columns = [ 'White', 'Mixed', 'Asian', 'BAC', 'Other', 'Deprivation'])
elif Data_Set == 3:
    test_data = pd.DataFrame(list(zip(test_GeoCode_Lat, test_GeoCode_Long, test_White, test_Mixed, test_Asian, test_BAC, test_Other, test_Deprivation)), columns = [ 'White', 'Mixed', 'Asian', 'BAC', 'Other', 'Deprivation'])
    
test_data.dropna(inplace=True)

test_data["RVM_Sector"]= test_data["RVM_Sector"].astype('float')
test_data["RVM_Area"]= test_data["RVM_Area"].astype('float')
test_data["RVM_Dist"]= test_data["RVM_Dist"].astype('float')
test_data["White"]= test_data["White"].astype('float')
test_data["Mixed"]= test_data["Mixed"].astype('float')
test_data["Asian"]= test_data["Asian"].astype('float')
test_data["BAC"]= test_data["BAC"].astype('float')
test_data["Other"]= test_data["Other"].astype('float')
test_data["Deprivation"]= test_data["Deprivation"].astype('float')

test_data["Business Type Index"]= test_data["Business Type Index"].astype('category')
#test_data = test_data.astype({col:value})


In [10]:
test_data.dtypes

Business Type Index    category
RVM_Sector              float64
RVM_Area                float64
RVM_Dist                float64
White                   float64
Mixed                   float64
Asian                   float64
BAC                     float64
Other                   float64
Deprivation             float64
dtype: object

In [11]:
y_pred = model.predict(test_data)

In [12]:
y_pred

array([4.55429775, 4.59193821, 4.57796344, 4.85183236, 4.24482157,
       4.00460195, 4.21726832, 3.77207679, 3.77373225, 4.24580597,
       4.64944254])

## Ultimate test

In [14]:

# load our data set and create 
Data = pd.read_csv("Data.csv")
Data["RVM_Sector"]= Data["RVM_Sector"].astype('float')
Data["RVM_Area"]= Data["RVM_Area"].astype('float')
Data["RVM_Dist"]= Data["RVM_Dist"].astype('float')
Data["White"]= Data["White"].astype('float')
Data["Mixed"]= Data["Mixed"].astype('float')
Data["Asian"]= Data["Asian"].astype('float')
Data["BAC"]= Data["BAC"].astype('float')
Data["Other"]= Data["Other"].astype('float')
Data["Deprivation"]= Data["Deprivation"].astype('float')
Data["Business Type Index"]= Data["Business Type Index"].astype('category')

X = Data.drop(['Business Name', 'Business Type', 'RatingValue', 'PostCode', 'RatingDate', 'LocalAuthorityName' ,'LocalAuthorityCode', 
                            'Hygiene', 'Structural', 'Confidence In Management', 'PostCode_S', 'PostCode', "pcd_sect","pcd_sect_S","Geocode_Lat", "Geocode_Long", "Postcode"] , axis=1)

y_pred = model.predict(X)



In [15]:
answer = pd.DataFrame(y_pred, columns = ["Predicted Result"])

In [17]:
result = pd.concat([Data, answer], axis = 1)

In [18]:
result

,Business Name,Business Type,PostCode,RatingValue,RatingDate,LocalAuthorityName,LocalAuthorityCode,Geocode_Lat,Geocode_Long,Hygiene,...,RVM_Area,RVM_Dist,White,Mixed,Asian,BAC,Other,Postcode,Deprivation,Predicted Result
0,1Oak Chilton Day Centre,Hospitals/Childcare/Caring Premises,CO10 2FT,5,2022-09-08,Babergh,297,52.044841,0.751574,0,...,4.775296,4.705128,0.970522,0.011791,0.009297,0.005669,0.002721,CO10 2FT,8.0,4.952309
1,A F C Cornard Dynamos Youth Football,Other catering premises,CO10 0NL,5,2018-03-26,Babergh,297,52.019962,0.753822,0,...,4.775296,4.705128,0.981459,0.007700,0.007327,0.002318,0.001196,CO10 0NL,6.0,4.950175
2,A G Lifestyle,Restaurant/Cafe/Canteen,CO10 2SS,5,2021-11-03,Babergh,297,52.036888,0.731603,5,...,4.775296,4.705128,0.970522,0.011791,0.009297,0.005669,0.002721,CO10 2SS,4.0,4.737605
3,A Leeder Butchers,Retailers - other,CO10 5NZ,5,2022-11-15,Babergh,297,52.029349,0.857566,5,...,4.775296,4.705128,0.990278,0.005745,0.002651,0.001326,0.000000,CO10 5NZ,7.0,4.402263
4,A Slice of New York,Restaurant/Cafe/Canteen,CO10 2AA,5,2022-11-01,Babergh,297,52.037352,0.729080,5,...,4.775296,4.705128,0.970522,0.011791,0.009297,0.005669,0.002721,CO10 2AA,4.0,4.737605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330606,Trigg Food Events,Mobile caterer,HR3 5RB,5,2022-06-21,Powys,566,52.071989,-3.118161,5,...,4.560000,4.333333,0.989500,0.004830,0.003780,0.001050,0.000840,HR3 5RB,5.0,5.076678
330607,Beth Bakes,Other catering premises,SY10 9JQ,5,2014-03-19,Powys,566,52.806916,-3.165747,0,...,4.611804,4.565517,0.987156,0.006215,0.005179,0.000414,0.001036,SY10 9JQ,4.0,4.888745
330608,Llanymynech Golf Club,Pub/bar/nightclub,SY10 8LB,5,2019-10-31,Powys,566,52.780861,-3.088750,5,...,4.611804,4.565517,0.990360,0.005671,0.003119,0.000284,0.000567,SY10 8LB,7.0,4.692578
330609,Training for Hire Ltd,Pub/bar/nightclub,SY10 9JQ,4,2018-01-30,Powys,566,52.806915,-3.165746,5,...,4.611804,4.565517,0.987156,0.006215,0.005179,0.000414,0.001036,SY10 9JQ,4.0,4.545171


In [19]:
summary = pd.pivot_table(result, index = ['LocalAuthorityName'], values = ["Predicted Result", "RatingValue"])

In [20]:
dummy = summary.to_csv("summary.csv")